In [10]:
import pandas as pd
import re

In [3]:
years = ['1999', '2001', '2003', '2005', '2007',
         '2009', '2011', '2013', '2015', '2017', 
         '2019', '2000', '2002', '2004', '2006',
         '2008', '2010', '2012', '2014', '2016', 
         '2018']

THERE'S SOME WEIRD STUFF LIKE #=72 IN 2018...GO BACK AND FIX THIS...

In [42]:
df_dict = {}
for eachYear in years:
    file_2019 = open('./raw_wiki_data/%s.txt' % str(eachYear), 'r')

    current_case = 0
    justice_info = {}
    justice_df = pd.DataFrame()
    for each_line in file_2019:
        # Getting the justices:
        if 'SCOTUS-termlist-start' in each_line:
            justices_info = each_line.strip().strip('}}').split('|')[1:]
            justices_info = [item.lower() for item in justices_info]
            justice_df = pd.DataFrame(columns=justices_info)

        # If you hit this, that means you're on a new case:
        if '#=' in each_line:
            # Updating the iterator:
            previous_case = current_case
            current_case += 1
            # If you don't have any justice info yet, continue
            if len(justice_info.keys()) == 0:
                continue
            # Otherwise, make a df out of that info:
            temp_df = pd.DataFrame.from_dict(justice_info)
            justice_df = justice_df.append(temp_df)
            # Reset the dict:
            justice_info = {}

        # If there's no justice info, fuggetaboutit
        if '|justice' not in each_line:
            continue

        # Otherwise...
        else:
            # Start by getting the justice:
            justice_list = each_line.split('--')
            justice = justice_list[1]
            # Get the opinion:
            opinion = justice_list[0].split('=')[1].split(' |')[0]
            justice_info[justice.lower()] = [opinion]

    justice_df.to_excel('./parsed_wiki_data/%s.xlsx' % str(eachYear))
    df_dict[eachYear] = justice_df

Right now, I'm doing splitting...instead, I'm going to get all possible strings, and pattern match:

In [43]:
all_combined = []
for each_year in df_dict.keys():
    for each_column in df_dict[each_year]:
        temp_column_val = list(set([re.sub(r'\d+', '', item) for item in list(set(df_dict[each_year][each_column]))]))
        all_combined = all_combined + temp_column_val

TypeError: expected string or bytes-like object

In [44]:
each_year

'2005'

In [45]:
each_column

"o'connor"

In [54]:
df_dict['2005']

,roberts,stevens,o'connor,scalia,kennedy,souter,thomas,ginsburg,breyer,alito,alito not yet appointed,o'connor retired
0,joinmajority,joinmajority,joinmajority,joinmajority,joinmajority,joinmajority,joinmajority,joinmajority,joinmajority,NaN,didnotparticipate,NaN
0,joinmajority,joinmajority,joinmajority,joinmajority,joinmajority,joinmajority,joinmajority,joinmajority,joinmajority,NaN,didnotparticipate,NaN
0,joinmajority,joinmajority,joinmajority,joinmajority,joinmajority,joinmajority,joinmajority,joinmajority,joinmajority,NaN,didnotparticipate,NaN
0,joinmajority,joinmajority,joinmajority,joinmajority,joinmajority,joinmajority,joinmajority,joinmajority,joinmajority,NaN,didnotparticipate,NaN
0,joinmajority,majority,joinmajority,joinmajority,joinmajority,joinmajority,joinmajority,joinmajority,joinmajority,NaN,didnotparticipate,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
0,joinmajority,joindissent2,NaN,joinmajority,joinmajority,dissent1,joinmajority,concurrence,dissent2,majority,NaN,didnotparticipate
0,majority,joindissent,NaN,joinmajority,joinmajority,joindissent,joinmajority,concurrence,dissent,joinmajority,NaN,didnotparticipate
0,partjoinmajority,partjoinmajority,NaN,concurrencedissent3,plurality,partjoinmajority,joinconcurrencedissent3,partjoinmajority,partjoinmajority,partjoinmajority,NaN,didnotparticipate
0,joinmajority,dissent1,NaN,joinconcurrence,joinmajority,joinmajority,concurrence,joindissent1,plurality,didnotparticipate,NaN,didnotparticipate


In [22]:
rob_val = list(set([re.sub(r'\d+', '', item) for item in list(set(justice_df['roberts']))]))
thomas_val = list(set([re.sub(r'\d+', '', item) for item in list(set(justice_df['thomas']))]))
ginsburg_val = list(set([re.sub(r'\d+', '', item) for item in list(set(justice_df['ginsburg']))]))
breyer_val = list(set([re.sub(r'\d+', '', item) for item in list(set(justice_df['breyer']))]))
alito_val = list(set([re.sub(r'\d+', '', item) for item in list(set(justice_df['alito']))]))
sotomayor_val = list(set([re.sub(r'\d+', '', item) for item in list(set(justice_df['sotomayor']))]))
kagan_val = list(set([re.sub(r'\d+', '', item) for item in list(set(justice_df['kagan']))]))
gorsuch_val = list(set([re.sub(r'\d+', '', item) for item in list(set(justice_df['gorsuch']))]))
kavanaugh_val = list(set([re.sub(r'\d+', '', item) for item in list(set(justice_df['kavanaugh']))]))

In [29]:
all_combined = []
all_combined = rob_val + thomas_val + ginsburg_val + breyer_val + alito_val + sotomayor_val + kagan_val + gorsuch_val + kavanaugh_val
all_combined = list(set(all_combined))
cleaned_all_combined = list(set([item.split('|')[0] for item in all_combined]))

In [30]:
cleaned_all_combined

['joinconcurrencedissent',
 'didnotparticipate',
 'joindissent',
 'dissent',
 'joinmajority',
 'concurrencedissentwithoutopinion',
 'joinconcurrence',
 'concurrencedissent',
 'concurrencewithoutopinion',
 'partjoinmajority',
 'plurality',
 'partjoindissent',
 'majority',
 'concurrence',
 'partjoinconcurrence']